In [74]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import warnings
from sklearn import set_config
set_config(display="diagram")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [75]:
gw_2018 = pd.read_csv('ground_water_quality_2018_post.csv')
gw_2019 = pd.read_csv('ground_water_quality_2019_post.csv')
gw_2020 = pd.read_csv('ground_water_quality_2020_post.csv')

In [76]:
gw_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sno               374 non-null    int64  
 1   district          374 non-null    object 
 2   mandal            374 non-null    object 
 3   village           374 non-null    object 
 4   lat_gis           374 non-null    float64
 5   long_gis          374 non-null    float64
 6   gwl               371 non-null    float64
 7   season            374 non-null    object 
 8   pH                374 non-null    float64
 9   E.C               374 non-null    int64  
 10  TDS               374 non-null    float64
 11  CO3               374 non-null    int64  
 12  HCO3              374 non-null    float64
 13  Cl                374 non-null    int64  
 14  F                 374 non-null    float64
 15  NO3               374 non-null    float64
 16  SO4               374 non-null    float64
 1

In [77]:
gw_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sno               364 non-null    int64  
 1   district          364 non-null    object 
 2   mandal            364 non-null    object 
 3   village           364 non-null    object 
 4   lat_gis           364 non-null    float64
 5   long_gis          364 non-null    float64
 6   gwl               359 non-null    float64
 7   season            364 non-null    object 
 8   pH                364 non-null    float64
 9   EC                364 non-null    int64  
 10  TDS               364 non-null    float64
 11  CO_-2             204 non-null    float64
 12  HCO_ -            364 non-null    int64  
 13  Cl -              364 non-null    int64  
 14  F -               364 non-null    float64
 15  NO3-              364 non-null    float64
 16  SO4-2             364 non-null    float64
 1

In [78]:
gw_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sno               368 non-null    int64  
 1   district          368 non-null    object 
 2   mandal            368 non-null    object 
 3   village           368 non-null    object 
 4   lat_gis           368 non-null    float64
 5   long_gis          368 non-null    float64
 6   gwl               365 non-null    float64
 7   season            368 non-null    object 
 8   Unnamed: 8        0 non-null      float64
 9   pH                368 non-null    object 
 10  E.C               368 non-null    int64  
 11  TDS               368 non-null    float64
 12  CO3               368 non-null    float64
 13  HCO3              368 non-null    int64  
 14  Cl                368 non-null    int64  
 15  F                 368 non-null    float64
 16  NO3               368 non-null    float64
 1

In [79]:
# Rename columns
new_column_names = {
    'CO_-2': 'CO',
    'HCO_ -': 'HCO',
    'Cl -':'Cl',
    'F -':'F',
    'NO3- ':'NO3',
    'SO4-2':'SO4'
}

gw_2019= gw_2019.rename(columns=new_column_names, inplace=True)


In [80]:
gw_df = pd.concat([gw_2018, gw_2019, gw_2020], ignore_index=True)


In [81]:
gw_df.head()

,sno,district,mandal,village,lat_gis,long_gis,gwl,season,pH,E.C,...,Na,K,Ca,Mg,T.H,SAR,Classification,RSC meq / L,Classification.1,Unnamed: 8
0,1,ADILABAD,Adilabad,Adilabad,19.668300,78.524700,5.09,postmonsoon 2018,8.28,745,...,49.0,4.0,48.0,38.896,279.934211,1.273328,C2S1,-1.198684,P.S.,NaN
1,2,ADILABAD,Bazarhatnur,Bazarhatnur,19.458888,78.350833,5.10,postmonsoon 2018,8.29,921,...,42.0,5.0,56.0,63.206,399.893092,0.913166,C3S1,-3.397862,P.S.,NaN
2,3,ADILABAD,Gudihatnoor,Gudihatnoor,19.525555,78.512222,4.98,postmonsoon 2018,7.69,510,...,45.0,2.0,24.0,38.896,219.934211,1.319284,C2S1,-0.398684,P.S.,NaN
3,4,ADILABAD,Jainath,Jainath,19.730555,78.640000,5.75,postmonsoon 2018,8.09,422,...,27.0,1.0,32.0,19.448,159.967105,0.928155,C2S1,0.000658,P.S.,NaN
4,5,ADILABAD,Narnoor,Narnoor,19.495665,78.852654,2.15,postmonsoon 2018,8.21,2321,...,298.0,5.0,56.0,92.378,519.843750,5.682664,C4S2,-4.396875,P.S.,NaN


In [82]:
# Find columns with null values
gw_df_with_null = gw_df.columns[gw_df.isnull().any()]

# Display columns with null values
print("Columns with null values:")
print(gw_df_with_null)

Columns with null values:
Index(['gwl', 'Unnamed: 8'], dtype='object')


In [83]:
gw_df['gwl'].value_counts()

3.95     5
3.98     4
2.70     4
3.59     3
3.80     3
        ..
25.08    1
2.52     1
8.63     1
14.39    1
4.92     1
Name: gwl, Length: 591, dtype: int64

In [84]:
mean_value = gw_df['gwl'].mean()
gw_df['gwl'].fillna(mean_value, inplace=True)

In [85]:
gw_df['Classification'].value_counts()

C3S1    462
C2S1    170
C4S1     53
C4S2     27
C3S2      9
C4S4      7
C3S3      6
C4S3      2
C1S1      2
O.G       2
C3S4      1
C2S2      1
Name: Classification, dtype: int64

In [86]:
# Drop rows with missing values
#gw_df_no_missing = gw_df.dropna()


In [87]:
X = gw_df.drop('Classification', axis=1)
y = gw_df['Classification']

In [90]:
# Encode categorical variables using LabelEncoder
label_encoders = {}
X_encoded = X.copy()
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Impute missing values with the most frequent value (mode)
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X_encoded)

# Create and fit a RandomForestClassifier to determine feature importances
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_imputed, y)

# Create a SelectFromModel instance to select features based on feature importances
sfm = SelectFromModel(clf, threshold='median')
X_new = sfm.fit_transform(X_imputed, y)

# Get the indices of the selected features
selected_indices = sfm.get_support(indices=True)

# Get the names of the selected features
selected_features = X_encoded.columns[selected_indices]

print("Selected Features:")
print(selected_features)

Selected Features:
Index(['E.C', 'TDS', 'HCO3', 'Cl', 'NO3 ', 'SO4', 'Na', 'K', 'Ca', 'Mg', 'T.H',
       'SAR', 'RSC  meq  / L'],
      dtype='object')


In [91]:
# Suppress the specific warning
warnings.filterwarnings(action='ignore', category=UserWarning, message="The least populated class in y has only 1 members")

X_selected = X[selected_features]

# Calculate class counts
class_counts = y.value_counts()

# Get the classes with more than one member
valid_classes = class_counts[class_counts > 1].index

# Filter the data and labels for valid classes
X_filtered = X_selected[y.isin(valid_classes)]
y_filtered = y[y.isin(valid_classes)]

# Create a StratifiedShuffleSplit instance with n_splits=1
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Create empty lists to store the splits
X_train_splits = []
X_test_splits = []
y_train_splits = []
y_test_splits = []

# Convert the filtered y labels to an array
y_filtered_array = y_filtered.values

# Perform the train-test split
for train_index, test_index in sss.split(X_filtered, y_filtered_array):
    X_train_splits.append(X_filtered.iloc[train_index])
    X_test_splits.append(X_filtered.iloc[test_index])
    y_train_splits.append(y_filtered_array[train_index])
    y_test_splits.append(y_filtered_array[test_index])

# Iterate over each split and perform the pipeline, grid search, and evaluation
for i in range(len(X_train_splits)):
    X_train = X_train_splits[i]
    X_test = X_test_splits[i]
    y_train = y_train_splits[i]
    y_test = y_test_splits[i]

    # Define a pipeline for preprocessing and modeling
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(max_iter=1000))
    ])

    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'model__C': [0.001, 0.01, 0.1, 1, 10],
        'model__penalty': ['l2']  # Use only 'l2' penalty
    }

    # Perform grid search with cross-validation
    grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', verbose=1)
    grid_search.fit(X_train, y_train)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Evaluate the best model on the test set
    y_pred_test = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_test)
    classification_rep = classification_report(y_test, y_pred_test, zero_division=1)

    print("Best Model:", best_model)
    print("Accuracy on Test Set:", accuracy)
    print("Classification Report:\n", classification_rep)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Model: Pipeline(steps=[('scaler', StandardScaler()),
                ('model', LogisticRegression(C=10, max_iter=1000))])
Accuracy on Test Set: 0.9256756756756757
Classification Report:
               precision    recall  f1-score   support

        C2S1       0.94      0.97      0.96        34
        C3S1       0.95      0.98      0.96        93
        C3S2       1.00      1.00      1.00         2
        C3S3       1.00      1.00      1.00         1
        C4S1       0.82      0.82      0.82        11
        C4S2       0.00      0.00      0.00         5
        C4S4       1.00      1.00      1.00         1
         O.G       1.00      0.00      0.00         1

    accuracy                           0.93       148
   macro avg       0.84      0.72      0.72       148
weighted avg       0.91      0.93      0.91       148

